# **CATBOOST**

Biblioteca: https://catboost.ai/en/docs/

In [1]:
# Intalação
#!pip install catboost

### **Abrindo os arquivos criados no Pre-Processamento**

In [2]:
# Importando a Biblioteca
import pickle

In [3]:
alvo           = pickle.load(open('heart.pkl',  'rb'))
previsores     = pickle.load(open('heart2.pkl', 'rb'))
previsores_esc = pickle.load(open('heart3.pkl', 'rb'))
previsores2    = pickle.load(open('heart4.pkl', 'rb'))
previsores3    = pickle.load(open('heart5.pkl', 'rb'))
previsores3_esc= pickle.load(open('heart6.pkl', 'rb'))

# RESUMO DO PRE-PROCESSAMENTO

**<font color='green'> alvo </font>** = variável que se pretende atingir (tem ou não doença cardíaca).

**<font color='green'> Previsores </font>** = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas manualmente, sem escalonar.

**<font color='green'> Previsores_esc </font>** = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas, escalonada.

**<font color='green'> Previsores2 </font>** = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas pelo labelencoder.

**<font color='green'> Previsores3 </font>** = conjunto de variáveis previsoras transformadas pelo labelencoder e onehotencoder, sem escalonar.

**<font color='green'> Previsores3_esc </font>** = conjunto de variáveis previsoras transformadas pelo labelencoder e onehotencoder escalonada.

## **BASE DE TREINO E TESTE**

In [4]:
# Importando a biblioteca
from sklearn.model_selection import train_test_split

**Parâmetros train_test_split:**   
- **arrays:** nomes dos atributos previsores e alvo.   
- **test_size:** tamanho em porcentagem dos dados de teste. default é none.   
- **train_size:** tamanho em porcentagem dos dados de treinamento.default é none.  
- **random_state:** nomeação de um estado aleatório.   
- **shuffle:** embaralhamento dos dados aleatórios. Associado com o random_state ocorre o mesmo embaralhamento sempre. Default é True.  
- **stratify:** Possibilidade de dividir os dados de forma estratificada. Default é None (nesse caso é mantido a proporção, isto é, se tem 30% de zeros e 70% de 1 no dataframe, na separação em treinamento e teste se manterá essa proporção).

In [5]:
# Separando teste e treino
x_treino, x_test, y_treino, y_test = train_test_split(previsores, alvo, test_size = 0.3, random_state=10)

In [6]:
# Shape de x
x_treino.shape, x_test.shape

((641, 11), (276, 11))

In [7]:
# Shape de y
y_treino.shape, y_test.shape

((641,), (276,))

# **CATBOOST**

In [8]:
# Import da Biblioteca
from catboost import CatBoostClassifier

In [9]:
# Import dos dados novamente para criamos um novo previsor
import pandas as pd

df = pd.read_csv('heart_tratado.csv',sep=';', encoding='utf-8')
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289.0,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180.0,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283.0,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214.0,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195.0,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
912,45,M,TA,110,264.0,0,Normal,132,N,1.2,Flat,1
913,68,M,ASY,144,193.0,1,Normal,141,N,3.4,Flat,1
914,57,M,ASY,130,131.0,0,Normal,115,Y,1.2,Flat,1
915,57,F,ATA,130,236.0,0,LVH,174,N,0.0,Flat,1


In [10]:
# Criando o previsor
previsores4 = df.iloc[:, 0:11]
previsores4.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,40,M,ATA,140,289.0,0,Normal,172,N,0.0,Up
1,49,F,NAP,160,180.0,0,Normal,156,N,1.0,Flat
2,37,M,ATA,130,283.0,0,ST,98,N,0.0,Up
3,48,F,ASY,138,214.0,0,Normal,108,Y,1.5,Flat
4,54,M,NAP,150,195.0,0,Normal,122,N,0.0,Up


In [11]:
# Criando o alvo
alvo4 = df.iloc[:, 11]
alvo4

0      0
1      1
2      0
3      1
4      0
      ..
912    1
913    1
914    1
915    1
916    0
Name: HeartDisease, Length: 917, dtype: int64

In [12]:
# Import da biblioteca de train e test
from sklearn.model_selection import train_test_split

x_treino, x_test, y_treino, y_test = train_test_split(previsores4, alvo4, test_size = 0.3, random_state=0)

In [13]:
# Separando as variaveis categoricas
categorias = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
categorias

['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

In [121]:
# Criando o classificador
catboost = CatBoostClassifier(task_type='CPU', iterations=100, learning_rate=0.1, depth=8, 
                              random_state=5, eval_metric='Accuracy')

In [122]:
# Treinando o modelo
catboost.fit(x_treino, y_treino, cat_features=categorias, plot=True, eval_set=(x_test, y_test))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8861154	test: 0.7789855	best: 0.7789855 (0)	total: 32.6ms	remaining: 3.22s
1:	learn: 0.8689548	test: 0.7934783	best: 0.7934783 (1)	total: 71.2ms	remaining: 3.49s
2:	learn: 0.8471139	test: 0.7971014	best: 0.7971014 (2)	total: 89.8ms	remaining: 2.9s
3:	learn: 0.8549142	test: 0.8043478	best: 0.8043478 (3)	total: 115ms	remaining: 2.77s
4:	learn: 0.8517941	test: 0.8043478	best: 0.8043478 (3)	total: 147ms	remaining: 2.8s
5:	learn: 0.8486739	test: 0.8043478	best: 0.8043478 (3)	total: 183ms	remaining: 2.86s
6:	learn: 0.8580343	test: 0.8115942	best: 0.8115942 (6)	total: 212ms	remaining: 2.82s
7:	learn: 0.8642746	test: 0.8043478	best: 0.8115942 (6)	total: 242ms	remaining: 2.79s
8:	learn: 0.8533541	test: 0.8043478	best: 0.8115942 (6)	total: 277ms	remaining: 2.81s
9:	learn: 0.8611544	test: 0.8115942	best: 0.8115942 (6)	total: 303ms	remaining: 2.73s
10:	learn: 0.8642746	test: 0.8079710	best: 0.8115942 (6)	total: 342ms	remaining: 2.77s
11:	learn: 0.8658346	test: 0.8043478	best: 0.8115942

97:	learn: 0.9282371	test: 0.8659420	best: 0.8695652 (54)	total: 2.66s	remaining: 54.3ms
98:	learn: 0.9282371	test: 0.8659420	best: 0.8695652 (54)	total: 2.69s	remaining: 27.2ms
99:	learn: 0.9297972	test: 0.8659420	best: 0.8695652 (54)	total: 2.71s	remaining: 0us

bestTest = 0.8695652174
bestIteration = 54

Shrink model to first 55 iterations.


In [123]:
# Criando o previsor
previsor_cat = catboost.predict(x_test)
previsor_cat[:100]

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0], dtype=int64)

In [124]:
# Comparando com y_test
print(list(y_test[:100]))

[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0]


In [125]:
# Import da biblioteca para verifica a acuracia e previsao
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [126]:
# Acuracia
print(f"Acuracia de Teste: {accuracy_score(y_test, previsor_cat)*100:.2f}%")

Acuracia de Teste: 86.96%


In [127]:
# Matrix de confusao
confusion_matrix(y_test, previsor_cat)

array([[103,  18],
       [ 18, 137]], dtype=int64)

In [128]:
# Resultado 
print(classification_report(y_test, previsor_cat))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85       121
           1       0.88      0.88      0.88       155

    accuracy                           0.87       276
   macro avg       0.87      0.87      0.87       276
weighted avg       0.87      0.87      0.87       276



**Analise dos dados de Treino**

In [129]:
previsores_treino = catboost.predict(x_treino)
previsores_treino[:100]

array([1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1], dtype=int64)

In [130]:
# Comparando com y_treino
print(list(y_treino[:100]))

[1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]


In [131]:
# Acuracia
print(f"Acuracia de Treino: {accuracy_score(y_treino, previsores_treino)*100:.2f}%")

Acuracia de Treino: 91.73%


In [132]:
# Matrix de confusao
confusion_matrix(y_treino, previsores_treino)

array([[255,  34],
       [ 19, 333]], dtype=int64)

In [133]:
# Resultado
print(classification_report(y_treino, previsores_treino))

              precision    recall  f1-score   support

           0       0.93      0.88      0.91       289
           1       0.91      0.95      0.93       352

    accuracy                           0.92       641
   macro avg       0.92      0.91      0.92       641
weighted avg       0.92      0.92      0.92       641



### Validação Cruzada

In [134]:
# Import das biliotecas
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [135]:
# Separando os dados em folds
kfold = KFold(n_splits=30, shuffle=True, random_state=5)

In [138]:
# Criando o Modelo
modelo = CatBoostClassifier(task_type='CPU', iterations=100, learning_rate=0.1, depth=8, random_state=5,
                           eval_metric='Accuracy')
resultado = cross_val_score(modelo, previsores, alvo, cv=kfold)

# Utilizando a Media e o Desvio padrao
print(f"Acuracia Media: {resultado.mean()*100:.2f}%")

0:	learn: 0.8803612	total: 5.33ms	remaining: 528ms
1:	learn: 0.8758465	total: 10.7ms	remaining: 524ms
2:	learn: 0.8837472	total: 19.9ms	remaining: 642ms
3:	learn: 0.8927765	total: 29.9ms	remaining: 716ms
4:	learn: 0.8792325	total: 31.1ms	remaining: 591ms
5:	learn: 0.8803612	total: 36.2ms	remaining: 568ms
6:	learn: 0.8893905	total: 41.5ms	remaining: 551ms
7:	learn: 0.8860045	total: 45.9ms	remaining: 528ms
8:	learn: 0.8927765	total: 50.1ms	remaining: 507ms
9:	learn: 0.8950339	total: 54.9ms	remaining: 494ms
10:	learn: 0.8950339	total: 59.2ms	remaining: 479ms
11:	learn: 0.8961625	total: 63.3ms	remaining: 464ms
12:	learn: 0.8972912	total: 67.6ms	remaining: 452ms
13:	learn: 0.8972912	total: 71.7ms	remaining: 441ms
14:	learn: 0.8995485	total: 76.2ms	remaining: 432ms
15:	learn: 0.9006772	total: 80.5ms	remaining: 422ms
16:	learn: 0.9029345	total: 84.5ms	remaining: 412ms
17:	learn: 0.9040632	total: 88.6ms	remaining: 404ms
18:	learn: 0.9018059	total: 89.6ms	remaining: 382ms
19:	learn: 0.9051919	t

63:	learn: 0.9277652	total: 287ms	remaining: 161ms
64:	learn: 0.9277652	total: 293ms	remaining: 158ms
65:	learn: 0.9277652	total: 297ms	remaining: 153ms
66:	learn: 0.9277652	total: 302ms	remaining: 149ms
67:	learn: 0.9288939	total: 307ms	remaining: 144ms
68:	learn: 0.9300226	total: 312ms	remaining: 140ms
69:	learn: 0.9311512	total: 319ms	remaining: 137ms
70:	learn: 0.9300226	total: 324ms	remaining: 132ms
71:	learn: 0.9311512	total: 329ms	remaining: 128ms
72:	learn: 0.9311512	total: 334ms	remaining: 124ms
73:	learn: 0.9300226	total: 340ms	remaining: 119ms
74:	learn: 0.9311512	total: 345ms	remaining: 115ms
75:	learn: 0.9300226	total: 349ms	remaining: 110ms
76:	learn: 0.9300226	total: 353ms	remaining: 106ms
77:	learn: 0.9300226	total: 358ms	remaining: 101ms
78:	learn: 0.9300226	total: 362ms	remaining: 96.3ms
79:	learn: 0.9300226	total: 366ms	remaining: 91.6ms
80:	learn: 0.9311512	total: 371ms	remaining: 86.9ms
81:	learn: 0.9300226	total: 375ms	remaining: 82.3ms
82:	learn: 0.9311512	total:

40:	learn: 0.9187359	total: 196ms	remaining: 282ms
41:	learn: 0.9187359	total: 201ms	remaining: 277ms
42:	learn: 0.9198646	total: 205ms	remaining: 272ms
43:	learn: 0.9198646	total: 210ms	remaining: 267ms
44:	learn: 0.9209932	total: 216ms	remaining: 264ms
45:	learn: 0.9232506	total: 222ms	remaining: 261ms
46:	learn: 0.9232506	total: 229ms	remaining: 258ms
47:	learn: 0.9232506	total: 234ms	remaining: 253ms
48:	learn: 0.9232506	total: 239ms	remaining: 249ms
49:	learn: 0.9243792	total: 243ms	remaining: 243ms
50:	learn: 0.9243792	total: 247ms	remaining: 237ms
51:	learn: 0.9243792	total: 252ms	remaining: 232ms
52:	learn: 0.9243792	total: 256ms	remaining: 227ms
53:	learn: 0.9243792	total: 260ms	remaining: 222ms
54:	learn: 0.9243792	total: 265ms	remaining: 217ms
55:	learn: 0.9255079	total: 269ms	remaining: 211ms
56:	learn: 0.9232506	total: 273ms	remaining: 206ms
57:	learn: 0.9232506	total: 277ms	remaining: 201ms
58:	learn: 0.9209932	total: 282ms	remaining: 196ms
59:	learn: 0.9243792	total: 286

16:	learn: 0.8995485	total: 83.9ms	remaining: 410ms
17:	learn: 0.9018059	total: 89.4ms	remaining: 407ms
18:	learn: 0.8939052	total: 90.4ms	remaining: 386ms
19:	learn: 0.8950339	total: 95.1ms	remaining: 380ms
20:	learn: 0.8984199	total: 100ms	remaining: 376ms
21:	learn: 0.8995485	total: 105ms	remaining: 374ms
22:	learn: 0.9006772	total: 113ms	remaining: 379ms
23:	learn: 0.9051919	total: 119ms	remaining: 376ms
24:	learn: 0.9074492	total: 125ms	remaining: 374ms
25:	learn: 0.9142212	total: 130ms	remaining: 371ms
26:	learn: 0.9051919	total: 132ms	remaining: 357ms
27:	learn: 0.9074492	total: 138ms	remaining: 355ms
28:	learn: 0.9085779	total: 143ms	remaining: 351ms
29:	learn: 0.9108352	total: 151ms	remaining: 352ms
30:	learn: 0.9153499	total: 156ms	remaining: 347ms
31:	learn: 0.9164786	total: 161ms	remaining: 341ms
32:	learn: 0.9187359	total: 166ms	remaining: 338ms
33:	learn: 0.9198646	total: 172ms	remaining: 333ms
34:	learn: 0.9209932	total: 176ms	remaining: 327ms
35:	learn: 0.9198646	total:

90:	learn: 0.9413093	total: 401ms	remaining: 39.6ms
91:	learn: 0.9413093	total: 406ms	remaining: 35.3ms
92:	learn: 0.9413093	total: 407ms	remaining: 30.6ms
93:	learn: 0.9413093	total: 411ms	remaining: 26.2ms
94:	learn: 0.9413093	total: 416ms	remaining: 21.9ms
95:	learn: 0.9413093	total: 421ms	remaining: 17.6ms
96:	learn: 0.9413093	total: 426ms	remaining: 13.2ms
97:	learn: 0.9413093	total: 436ms	remaining: 8.89ms
98:	learn: 0.9413093	total: 441ms	remaining: 4.45ms
99:	learn: 0.9413093	total: 446ms	remaining: 0us
0:	learn: 0.8724605	total: 4.98ms	remaining: 493ms
1:	learn: 0.8803612	total: 9.3ms	remaining: 456ms
2:	learn: 0.8916479	total: 13.3ms	remaining: 431ms
3:	learn: 0.8905192	total: 17.3ms	remaining: 415ms
4:	learn: 0.8792325	total: 18.3ms	remaining: 347ms
5:	learn: 0.8893905	total: 22.8ms	remaining: 356ms
6:	learn: 0.8950339	total: 27.5ms	remaining: 366ms
7:	learn: 0.9063205	total: 38ms	remaining: 437ms
8:	learn: 0.8995485	total: 45ms	remaining: 455ms
9:	learn: 0.9029345	total: 49

...

57:	learn: 0.9267193	total: 439ms	remaining: 318ms
58:	learn: 0.9278467	total: 446ms	remaining: 310ms
59:	learn: 0.9278467	total: 457ms	remaining: 305ms
60:	learn: 0.9289741	total: 463ms	remaining: 296ms
61:	learn: 0.9289741	total: 470ms	remaining: 288ms
62:	learn: 0.9278467	total: 472ms	remaining: 277ms
63:	learn: 0.9278467	total: 481ms	remaining: 271ms
64:	learn: 0.9278467	total: 486ms	remaining: 262ms
65:	learn: 0.9301015	total: 495ms	remaining: 255ms
66:	learn: 0.9301015	total: 501ms	remaining: 247ms
67:	learn: 0.9323563	total: 505ms	remaining: 238ms
68:	learn: 0.9312289	total: 514ms	remaining: 231ms
69:	learn: 0.9334837	total: 520ms	remaining: 223ms
70:	learn: 0.9334837	total: 531ms	remaining: 217ms
71:	learn: 0.9334837	total: 537ms	remaining: 209ms
72:	learn: 0.9334837	total: 548ms	remaining: 203ms
73:	learn: 0.9346110	total: 553ms	remaining: 194ms
74:	learn: 0.9346110	total: 561ms	remaining: 187ms
75:	learn: 0.9346110	total: 567ms	remaining: 179ms
76:	learn: 0.9346110	total: 575

40:	learn: 0.9165727	total: 215ms	remaining: 309ms
41:	learn: 0.9177001	total: 222ms	remaining: 306ms
42:	learn: 0.9177001	total: 226ms	remaining: 300ms
43:	learn: 0.9177001	total: 231ms	remaining: 295ms
44:	learn: 0.9199549	total: 242ms	remaining: 295ms
45:	learn: 0.9210823	total: 248ms	remaining: 291ms
46:	learn: 0.9199549	total: 255ms	remaining: 288ms
47:	learn: 0.9210823	total: 260ms	remaining: 282ms
48:	learn: 0.9210823	total: 265ms	remaining: 276ms
49:	learn: 0.9210823	total: 270ms	remaining: 270ms
50:	learn: 0.9222097	total: 274ms	remaining: 263ms
51:	learn: 0.9233371	total: 279ms	remaining: 257ms
52:	learn: 0.9222097	total: 284ms	remaining: 252ms
53:	learn: 0.9233371	total: 289ms	remaining: 246ms
54:	learn: 0.9233371	total: 293ms	remaining: 240ms
55:	learn: 0.9255919	total: 299ms	remaining: 235ms
56:	learn: 0.9255919	total: 303ms	remaining: 229ms
57:	learn: 0.9267193	total: 308ms	remaining: 223ms
58:	learn: 0.9278467	total: 312ms	remaining: 216ms
59:	learn: 0.9289741	total: 319

9:	learn: 0.8726043	total: 40ms	remaining: 360ms
10:	learn: 0.8714769	total: 46.6ms	remaining: 377ms
11:	learn: 0.8726043	total: 51.2ms	remaining: 375ms
12:	learn: 0.8771139	total: 56.6ms	remaining: 379ms
13:	learn: 0.8793687	total: 63.8ms	remaining: 392ms
14:	learn: 0.8771139	total: 71ms	remaining: 402ms
15:	learn: 0.8793687	total: 79.4ms	remaining: 417ms
16:	learn: 0.8793687	total: 80.3ms	remaining: 392ms
17:	learn: 0.8793687	total: 85.1ms	remaining: 388ms
18:	learn: 0.8804961	total: 91.1ms	remaining: 388ms
19:	learn: 0.8838782	total: 95.2ms	remaining: 381ms
20:	learn: 0.8838782	total: 103ms	remaining: 386ms
21:	learn: 0.8861330	total: 109ms	remaining: 388ms
22:	learn: 0.8883878	total: 116ms	remaining: 389ms
23:	learn: 0.8917700	total: 124ms	remaining: 393ms
24:	learn: 0.8951522	total: 131ms	remaining: 394ms
25:	learn: 0.8951522	total: 137ms	remaining: 391ms
26:	learn: 0.8928974	total: 139ms	remaining: 376ms
27:	learn: 0.8951522	total: 144ms	remaining: 370ms
28:	learn: 0.8951522	tota

85:	learn: 0.9379932	total: 489ms	remaining: 79.7ms
86:	learn: 0.9391206	total: 499ms	remaining: 74.6ms
87:	learn: 0.9391206	total: 504ms	remaining: 68.7ms
88:	learn: 0.9402480	total: 511ms	remaining: 63.1ms
89:	learn: 0.9413754	total: 518ms	remaining: 57.6ms
90:	learn: 0.9402480	total: 524ms	remaining: 51.8ms
91:	learn: 0.9425028	total: 531ms	remaining: 46.2ms
92:	learn: 0.9425028	total: 536ms	remaining: 40.4ms
93:	learn: 0.9436302	total: 544ms	remaining: 34.7ms
94:	learn: 0.9436302	total: 549ms	remaining: 28.9ms
95:	learn: 0.9436302	total: 551ms	remaining: 23ms
96:	learn: 0.9436302	total: 556ms	remaining: 17.2ms
97:	learn: 0.9436302	total: 565ms	remaining: 11.5ms
98:	learn: 0.9436302	total: 576ms	remaining: 5.82ms
99:	learn: 0.9436302	total: 582ms	remaining: 0us
0:	learn: 0.8714769	total: 5.53ms	remaining: 547ms
1:	learn: 0.8793687	total: 9.76ms	remaining: 478ms
2:	learn: 0.8850056	total: 14.8ms	remaining: 479ms
3:	learn: 0.8883878	total: 19.8ms	remaining: 476ms
4:	learn: 0.8782413	t

53:	learn: 0.9289741	total: 309ms	remaining: 263ms
54:	learn: 0.9278467	total: 314ms	remaining: 257ms
55:	learn: 0.9278467	total: 319ms	remaining: 251ms
56:	learn: 0.9278467	total: 326ms	remaining: 246ms
57:	learn: 0.9267193	total: 337ms	remaining: 244ms
58:	learn: 0.9267193	total: 344ms	remaining: 239ms
59:	learn: 0.9267193	total: 351ms	remaining: 234ms
60:	learn: 0.9267193	total: 355ms	remaining: 227ms
61:	learn: 0.9255919	total: 360ms	remaining: 220ms
62:	learn: 0.9267193	total: 364ms	remaining: 214ms
63:	learn: 0.9278467	total: 369ms	remaining: 207ms
64:	learn: 0.9278467	total: 373ms	remaining: 201ms
65:	learn: 0.9278467	total: 377ms	remaining: 194ms
66:	learn: 0.9289741	total: 382ms	remaining: 188ms
67:	learn: 0.9312289	total: 386ms	remaining: 182ms
68:	learn: 0.9323563	total: 390ms	remaining: 175ms
69:	learn: 0.9334837	total: 395ms	remaining: 169ms
70:	learn: 0.9334837	total: 400ms	remaining: 163ms
71:	learn: 0.9334837	total: 404ms	remaining: 157ms
72:	learn: 0.9334837	total: 410